## Mod 2 Assessment
By Anthony Schams

In [48]:
# put all imports here
import json
import requests
import pandas as pd
import pymongo
import sqlite3
from bs4 import BeautifulSoup
from bson.json_util import loads
from collections import Counter

### SQL (10 Minutes)
There is a sqlite3 database in `assets/books.db`. The SQL to create this is also in `assets/books.sql`, if you want to run it manually (you can also import this using https://sqliteonline.com/, or run the SQL file directly).  Both have the same schema and data.

The schema has three tables. You can explore the schema in the fiddle posted above.  Please answer the following questions.

1. Connect to the database using sqlite3

In [4]:
# Your code here
conn = sqlite3.connect('assets/books.db')
c = conn.cursor()

2. Querying the DB: how many pages are in the book "Nine Stories"?

In [15]:
# Your code here
sql = """SELECT pages FROM book
         WHERE title = 'Nine Stories'"""
c.execute(sql).fetchone()[0]

600

3. How many authors are from the USA?

In [16]:
# Your code here
sql2 = """SELECT COUNT(*) FROM author
          WHERE country = 'USA'"""
c.execute(sql2).fetchone()[0]

6

4. How many authors does the book "Professional ASP.NET 4.5 in C# and VB" have?

In [17]:
# Your code here
sql3 = """SELECT COUNT(*) 
          FROM book b
          JOIN book_author ba
          USING(book_id)
          WHERE b.title = 'Professional ASP.NET 4.5 in C# and VB'"""
c.execute(sql3).fetchone()[0]

5

5. How many pages total have been written by non-American authors?

In [18]:
# Your code here
sql4 = """SELECT SUM(pages)
          FROM author a
          JOIN book_author ba
          USING(author_id)
          JOIN book b
          USING(book_id)
          WHERE a.country != 'USA'"""
c.execute(sql4).fetchone()[0]

30003

### Object Oriented Programming (15 Minutes)
Below is the stub of some classes. Make the classes adhere to the following rules. The code beneath the cell should  run successfully.
1. Force every new instance of `WeWorkMember` to pass in a value to set the attribute `name`.
1. Give every new instance of `WeWorkMember` an attribute `caffeinated` that is set to `False`. 
1. Give `WeWorkMember` an **instance** method called `caffeinate()` that prints out "Getting coffee!" and sets the attribute `caffeinated` to `True`.
1. Make it so that `Staff` and `Student` inherit from `WeWorkMember`
1. Give `Staff` a **static** method called `cheer()` that prints out "Goooooooo Flatiron Students!"
1. Give `Students` a **class** method called `learn()` that takes in an integer and returns that number +1

In [87]:
# Modify this cell
class WeWorkMember():
    
    def __init__(self, name):
        self.name = name
        self.caffeinated = False
    
    def caffeinate(self):
        print("Getting coffee!")
        self.caffeinated = True

class Staff(WeWorkMember):
    
    @staticmethod
    def cheer():
        print("Goooooooo Flatiron Students!")

class Student(WeWorkMember):
    
    @classmethod
    def learn(cls, num):
        return(num + 1)

In [88]:
# DO NOT MODIFY this cell. if you want to add more tests, add them in a new cell!
try:
    nobody = Staff()
    raise RuntimeError('Staff needs a name!')
except TypeError:
    pass

andy = Staff('Andy')
assert andy.caffeinated is False
andy.caffeinate()
assert andy.caffeinated is True

Staff.cheer()

bobby = Student('Bobby Tables')
assert bobby.caffeinated is False
bobby.caffeinate()
assert bobby.caffeinated is True

Student.learn(99)

Getting coffee!
Goooooooo Flatiron Students!
Getting coffee!


100

### APIs (10 Minutes)
Using this API (http://www.nokeynoshade.party/api/judges) about RuPaul's Drag Race (docs: https://drag-race-api.readme.io/docs/get-all-judges), tell me how many judges of each **`type`** there were in the first 50 records the API returns.
 - the 50 judges will have **`id`** ranging from 1 to 53
 - you can do the aggregation in pure Python, Pandas, SQL-- whatever's easiest for you

In [31]:
req = requests.get("http://www.nokeynoshade.party/api/judges", params={"limit":50})

In [41]:
rp_json = req.json()

In [46]:
judge_types = []
for ele in rp_json:
    judge_types.append(ele['type'])

In [49]:
Counter(judge_types)

Counter({'regular': 2, 'guest': 47, 'interim': 1})

### Web Scraping (15 Minutes)
Scrape the website [http://quotes.toscrape.com/](http://quotes.toscrape.com/).  Turn each quote on the page into a Pandas dataframe consisting of two columns: the author of the quote and the quote itself. The final result should be a dataframe of 10 rows (not counting header) and 2 columns (not counting index).

In [71]:
quotes_html = requests.get('http://quotes.toscrape.com/')


In [72]:
soup = BeautifulSoup(quotes_html.text, 'html.parser')

In [79]:
quotes = soup.findAll(class_='text', text=True)
authors = soup.findAll(class_='author', text=True)

In [83]:
quotes_list = []
for q in quotes:
    quotes_list.append(q.text)
    
authors_list = []
for a in authors:
    authors_list.append(a.text)

In [86]:
quote_df = pd.DataFrame(list(zip(quotes_list, authors_list)), columns=['Quote', 'Author'])
quote_df

,Quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein
6,“It is better to be hated for what you are tha...,André Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


### NoSQL (10 Minutes)
Answer all of the following questions by querying Mongo and manipulating the results in Python
1. Load data from `assets/grades.json` into Mongo (this code is written for you)

In [50]:
# you shouldn't need to edit this cell!
with open('assets/grades.jsonl') as f:
    # loads() comes from the bson library
    file_data = [loads(line) for line in f.readlines()]

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mod2db"]
coll = db["testcoll"]

coll.insert_many(file_data)

2. How many records are there total?

In [57]:
# Your code here
coll.count_documents(filter={})


280

3. How many students have taken the class with `class_id` = **29**?

In [58]:
# Your code here
coll.count_documents(filter={'class_id':29})

9

4. Super bonus question: for student **12** in class **23**, what grade did they get on their exam?

In [89]:
# Your code here
st12_cl23_scores = coll.find_one({'$and':[
                       {"class_id":23}, 
                       {"student_id":12}]}, {'scores':1})

In [92]:
scores_df = pd.DataFrame(st12_cl23_scores['scores'])

In [95]:
scores_df[scores_df.type == 'exam']['score']

0    26.985722
Name: score, dtype: float64

In [96]:
# for when you're done with this portion, this deletes the data we added.
client.drop_database("mod2db")

## Assessment submission (2 Minutes)
Please save your completed file as `mod2-assessment.ipynb` and upload it using [this form](https://docs.google.com/forms/d/e/1FAIpQLSczNVM9Q6Dr3w1YR3kkg4PNaOsfDO-qIvh1wLy6rR0SEwTwIQ/viewform?usp=sf_link)